In [134]:
import re
import pandas as pd
import numpy as np
import string
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

from datetime import datetime

import pickle 
import pyLDAvis
import pyLDAvis.gensim

import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en.stop_words import STOP_WORDS
import en_core_web_sm

import gensim
from gensim import corpora
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess
from gensim.models.ldamodel import LdaModel

from wordcloud import WordCloud
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams
from pprint import pprint

from tqdm import tqdm_notebook as tqdm_note
import tqdm


import warnings
warnings.filterwarnings('ignore')

# Import the document list

In [175]:
#import the doc_list file
#load the pickle 
filename= 'doc_spacyall'
infile= open(filename,'rb')
load_doc_spacyall = pickle.load(infile)
infile.close()

In [176]:
#import the doc_list file
#load the pickle 
filename= 'doc_nltk_bitri'
infile= open(filename,'rb')
load_doc_nltk_bitri = pickle.load(infile)
infile.close()

In [177]:
#import the doc_list file
#load the pickle 
filename= 'doc_clean_tweet'
infile= open(filename,'rb')
load_doc_clean_tweet = pickle.load(infile)
infile.close()

#### Cautions : This notebook takes a long time to Run. Thus I will not restart the kernel   

## Before buliding the LDA Model,I am trying to search for the best K topics by refering to coherence score.

In [139]:
#Function for LDA model, 
#pass the corpus with different pre-proccessing, number of topics, and number of iterition(passes)

def lda_model_fuc (filename, k, n):
    
    lda_model = gensim.models.ldamodel.LdaModel(corpus=filename,
                                               id2word=words,
                                               num_topics=k, 
                                               random_state=2,
                                               update_every=1,
                                               passes=n,
                                               alpha='auto',
                                               per_word_topics=True)
    pprint(lda_model.print_topics(num_words=10))


In [140]:
#Function for conference score
#help with determine the number of topics.

def coherence (k,n,load_doc_name):
    words = corpora.Dictionary(load_doc_name)
    corpus = [words.doc2bow(doc) for doc in load_doc_name]
    lda_model = LDA(corpus=corpus, id2word=words, num_topics=k, random_state=100,
                chunksize=200, passes=n, per_word_topics=True)

    coherence_model_lda = CoherenceModel(model=lda_model, texts=load_doc_name, dictionary=words, coherence='c_v')
    coherence_lda = round(coherence_model_lda.get_coherence(),4)
    print(f'Coherence Score of {k} Topics: {(coherence_lda)}')

## Comparing the coherence score of pre-processing `SpaCy pipelines` for 3-10 number of topics 

#### caution: the comparison of the coherence score take long time to run

In [141]:
#set the number of iterition(passes) to 20 
for num_topic in tqdm_note(range(3,11)):
    coherence (num_topic,20,load_doc_spacyall)

Coherence Score of 3 Topics: 0.3442
Coherence Score of 4 Topics: 0.3519
Coherence Score of 5 Topics: 0.3295
Coherence Score of 6 Topics: 0.3491
Coherence Score of 7 Topics: 0.3194
Coherence Score of 8 Topics: 0.309
Coherence Score of 9 Topics: 0.2965
Coherence Score of 10 Topics: 0.315



In [142]:
#set the number of iterition(passes) to 10 
for num_topic in tqdm_note(range(3,11)):
    coherence (num_topic,10,load_doc_spacyall)

Coherence Score of 3 Topics: 0.3455
Coherence Score of 4 Topics: 0.3519
Coherence Score of 5 Topics: 0.3295
Coherence Score of 6 Topics: 0.3491
Coherence Score of 7 Topics: 0.3176
Coherence Score of 8 Topics: 0.3065
Coherence Score of 9 Topics: 0.2965
Coherence Score of 10 Topics: 0.3147



## Comparing the coherence score of pre-processing `NLTK &Bigram/Trigram` for 3-10 number of topics 

#### caution: the comparison of the coherence score take long time to run

In [ ]:
##set the number of iterition(passes) to 20

#for num_topic in tqdm_note(range(3,11)):
 #   coherence (num_topic,20,load_doc_nltk_bitri)

Here I will only run for 10 passes as we can see that the score with passes 10 times and the score with passes 20 are almost the same.

In [143]:
#set the number of iterition(passes) to 10 

for num_topic in tqdm_note(range(3,11)):
    coherence (num_topic,10,load_doc_nltk_bitri)

Coherence Score of 3 Topics: 0.3515
Coherence Score of 4 Topics: 0.3405
Coherence Score of 5 Topics: 0.3369
Coherence Score of 6 Topics: 0.3549
Coherence Score of 7 Topics: 0.3201
Coherence Score of 8 Topics: 0.3251
Coherence Score of 9 Topics: 0.3117
Coherence Score of 10 Topics: 0.3254



## Comparing the coherence score of pre-processing `tweet cleaning` for 3-10 number of topics 

#### caution: the comparing of the coherence score take long time to run

In [61]:
#set the number of iterition(passes) to 20 

#for num_topic in tqdm_note(range(3,11)):
 #   coherence (num_topic,20,load_doc_clean_tweet)

Coherence Score of 3 Topics: 0.3568536642512174
Coherence Score of 4 Topics: 0.3133538450797512
Coherence Score of 5 Topics: 0.32682706598055083
Coherence Score of 6 Topics: 0.2956369565190991
Coherence Score of 7 Topics: 0.3442207300846051
Coherence Score of 8 Topics: 0.331717454078419
Coherence Score of 9 Topics: 0.32139580483673746
Coherence Score of 10 Topics: 0.3270063066711949


Here I will only run for 10 passes as we can see that the score with passes 10 times and the score with passes 20 are almost the same.

In [144]:
# #set the number of iterition(passes) to 10 

for num_topic in tqdm_note(range(3,11)):
    coherence (num_topic,10,load_doc_clean_tweet)

Coherence Score of 3 Topics: 0.3688
Coherence Score of 4 Topics: 0.3353
Coherence Score of 5 Topics: 0.336
Coherence Score of 6 Topics: 0.3142
Coherence Score of 7 Topics: 0.3207
Coherence Score of 8 Topics: 0.304
Coherence Score of 9 Topics: 0.3123
Coherence Score of 10 Topics: 0.296



__Short summary of the above coherence score__

Overall the coherence score are low. The highest score is 0.3688 tweet cleaning with 3 topics
The passess didn't affect the coherence score as we can see that the score with passes 10 times and the score with passes 20 are almost the same.

It's possible to tune for the parameter k , **number of topics**, though it's not necessarily a very straightforward way to improve the model, I will try to tune the hyperparameter aiming at getting a higher coherence score.

### Tuning hyperparameter : alpha and eta(beta) are hyperparameters that affect sparsity of the topics. 

__Tuning LDA model can be tough__

More legitimate implementations of LDA have hyperparameters such as alpha. Alpha is a scaler that helps minimize an error term. Most LDA models that are implented will set this automatically and it's usually, 95% of the time a fine solution. 

As the coherence scores above are not ideal with auto alpha, I will perform a series of sensitivity tests to help determine the following model hyperparameters:
- Number of Topics (K)
- Dirichlet hyperparameter alpha: Document-Topic Density
- Dirichlet hyperparameter beta: Word-Topic Density

Alpha parameter is Dirichlet prior concentration parameter that represents document-topic density — with a higher alpha, documents are assumed to be made up of more topics and result in more specific topic distribution per document.

Beta parameter is the same prior concentration parameter that represents topic-word density — with high beta, topics are assumed to made of up most of the words and result in a more specific word distribution per topic.

In [75]:
# function for tunning coherence score
# Here i will set the passes = 20 because the coherence score are 
def coherence_tuning(corpus, load_doc_name, k, a, b):
    words = corpora.Dictionary(load_doc_name)
    corpus = [words.doc2bow(doc) for doc in load_doc_name]
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=words,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=20,
                                           alpha=a,
                                           eta=b,
                                           per_word_topics= True)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=load_doc_name, dictionary=words, coherence='c_v')
    
    #return coherence_model_lda.get_coherence()
    coherence_lda = coherence_model_lda.get_coherence()
    print(f'Coherence Score of {k} Topics: {coherence_lda}')

In [83]:
grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 3
max_topics = 6
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.1, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.1, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [corpus]

corpus_title = ['100% Corpus']

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

#### Cautions: the cell below take 14 hours to run

In [84]:
# Can take a long time to run
#I run this cell only once and will not run it again as it takes a very long time
if 1 == 1:
    pbar = tqdm.tqdm(total=(len(beta)*len(alpha)*len(topics_range)*len(corpus_title)))
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = coherence_tuning(corpus,load_doc_spacyall, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results.csv', index=False)
    pbar.close()


  2%|▏         | 1/60 [12:40<12:27:51, 760.53s/it]

Coherence Score of 3 Topics: 0.31233030049287835



  3%|▎         | 2/60 [25:13<12:12:57, 758.23s/it]

Coherence Score of 3 Topics: 0.29071935193145293



  5%|▌         | 3/60 [37:58<12:02:22, 760.40s/it]

...

Coherence Score of 5 Topics: 0.4325252149550566



 98%|█████████▊| 59/60 [13:43:47<14:37, 877.70s/it]

Coherence Score of 5 Topics: 0.4308405576536586



100%|██████████| 60/60 [13:58:22<00:00, 838.37s/it]

Coherence Score of 5 Topics: 0.3283152261973334


In [88]:
#loading the saved csv as dataframe
lda_tunning = pd.read_csv('lda_tuning_results.csv')

In [147]:
#finding the best hyperparameters
lda_tunning.iloc[34]

Validation_Set           100% Corpus
Topics                             4
Alpha                      symmetric
Beta              0.7000000000000001
Coherence                        NaN
Name: 34, dtype: object

In [94]:
#checking the best with the best hyperparaters
best_score = coherence_tuning(corpus,load_doc_spacyall,k=4, a='symmetric', b=0.7000000000000001)

Coherence Score of 4 Topics: 0.4400192893615461


#### __Summary:__
From the tuning process , I make a mistake that I set the output of `function of coherence tunning `( as ` print(f'Coherence Score of {k} Topics: {coherence_lda}'` , this is a string and that is why the value of coherence score on cvs file named `lda_tuning_results` become NaN. 
However, I am not going to rerun the cell as it takes too long time to run.

I can find the hyperparameters (number of topic is 4, alpha is symmetric, beta is 0.7000000000000001) of best score by looking at the output below, the best score after tunning is around 0.45.
It is on the index 35 of the dataframe lda_tunning.

Coherence Score of 4 Topics: 0.4469748529454709
 58%|█████▊    | 35/60 [7:54:32<5:42:42, 822.50s/it]
 

This time I only run for one input `load_doc_spacyall` due to the long running time. Originally I plan to tune the other 2 inputs `load_doc_nltk_bitri` and `load_doc_clean_tweet` as well.

Since the running time takes long time, I will first set 4 topics, alpha is auto and beta is none for each input. I will determine which input to use with the best hyperparameter. 

